In [1]:
import pandas as pd
import json

In [2]:
# Constants :

propertyMapper = {
    'township': {
        'unknown': 'Inconnue',
        'paris01': 'PARIS 01',
        'paris02': 'PARIS 02',
        'paris03': 'PARIS 03',
        'paris04': 'PARIS 04',
        'paris05': 'PARIS 05',
        'paris06': 'PARIS 06',
        'paris07': 'PARIS 07',
        'paris08': 'PARIS 08',
        'paris09': 'PARIS 09',
        'paris10': 'PARIS 10',
        'paris11': 'PARIS 11',
        'paris12': 'PARIS 12',
        'paris13': 'PARIS 13',
        'paris14': 'PARIS 14',
        'paris15': 'PARIS 15',
        'paris16': 'PARIS 16',
        'paris17': 'PARIS 17',
        'paris18': 'PARIS 18',
        'paris19': 'PARIS 19',
        'paris20': 'PARIS 20',
        'val_de_marne': 'Val-de-Marne',
        'yvelines': 'Yvelines',
        'seine_et_marne': 'Seine-et-Marne',
        'val_d_oise': "Val d'Oise",
        'hauts_de_seine': 'Hauts-de-Seine',
        'essonne': 'Essonne',
        'seine_saint_denis': "Seine-Saint-Denis"
    },
    'year': {
        '2013': '2013',
        '2014': '2014',
        '2015': '2015',
        '2016': '2016',
        '2017': '2017',
        '2018': '2018',
        '2019': '2019',
        '2020': '2020',
        '2021': '2021',
        '2022': '2022',
        '2023': '2023',
    },
    'city_committee': {
        'DDCT': 'DDCT', 'DAC': 'DAC', 'DJS': 'DJS', 'DASES': 'DASES', 'DAE': 'DAE', 'DFPE': 'DFPE', 'DASCO': 'DASCO', 'DPSP': 'DPSP', 'DPVI': 'DPVI',
        'DGRI': 'DGRI', 'DEVE': 'DEVE', 'DSOL': 'DSOL', 'DPMP': 'DPMP', 'DPE': 'DPE', 'DSP': 'DSP', 'DVD': 'DVD', 'SG': 'SG', 'DUCT': 'DUCT',
        'DGOM': 'DGOM', 'DLH': 'DLH', 'SG-MI-CINEMA': 'SG-MI-CINEMA', 'DU': 'DU', 'DTEC': 'DTEC', 'DRH': 'DRH', 'DFA': 'DFA', 'DICOM': 'DICOM',
        'CASVP': 'CASVP', 'DAJ': 'DAJ', 'DILT': 'DILT', 'SG-DPMC': 'SG-DPMC', 'SGCP': 'SGCP'
    },
    'activity': {
        'culture_arts': 'Culture & Arts', 'education_formation': 'Education & formation',
        'rights_interests_defense': 'Défense des droits et des intérêts', 'hobbies': 'Loisirs', 'social': 'Social',
        'precarity_exclusion': 'Précarité & Exclusion', 'association_help': 'Aides aux associations', 'sport': 'Sport', 'work': 'Emploi',
        'communication_media': 'Communication & média', 'humanitarian': 'Humanitaire', 'economy': 'Economie',
        'environment_ecology': 'Environnement & écologie', 'health': 'Santé', 'local_life': 'Vie et animation locale',
        'architecture_urban_planning': 'Architecture & urbanisme', 'memory': 'Mémoire', 'travel_means': 'Déplacements et transports',
        'international_relationships': 'Relations internationales', 'ideas_opinions': 'Idée & opinion', 'tourism': 'Tourisme',
        'technology_research': 'Technique & Recherche'
    },
    'aim_subvention': {
        'not_specified': 'Non précisée', 'project': 'Projet', 'operation': 'Fonctionnement', 'investment': 'Investissement'
    }
}

In [3]:
# Helper functions :

        # Creating a new file with selected columns streaming from a file via a file descriptor :

def createFileWithSpecificColumns(origin, destination):
    with open(origin) as b, open(destination,'w', encoding = 'utf-8') as e:
        line = b.readline()
        while line!='':
            splittedLine = line.split(',')
            siret = splittedLine[2]
            postalCode = splittedLine[16]
            e.write(f'{siret},{postalCode}' +"\n")
            line = b.readline()

        # Checking data validity :

def checkFalseDuplicates(x, year):
    data_tmp = x[x['Année budgétaire'] == year]
    
    def check(row):
        file_id=row['Numéro de dossier']
        same_file_id_row_amount = data_tmp[data_tmp['Numéro de dossier'] == file_id]['Montant voté']
        result = (same_file_id_row_amount == row['Montant voté']).value_counts()
        if False in result.index:
            return False
        else :
            return True
    return data_tmp.apply(check, axis=1)

        # Transforming data types :

def transformOriginDataTypesIntoFinalDataTypes(dataset, originDTypes, finalDTypes):
    for (originDType, finalDType) in zip(originDTypes, finalDTypes):
        for column in dataset.dtypes[dataset.dtypes == originDType].index:
            dataset[column] = dataset[column].astype(finalDType)
    return dataset

def transformSelectedColumnsDataTypeIntoSpecifiedDataType(dataset, columnsDataTypeMapping):
    for column, dtype in columnsDataTypeMapping.items():
        dataset[column] = dataset[column].astype(dtype)
    return dataset
    
        #Selecting (or filtering) data:

def selectingRowsWithSpecificValuesInSpecifiedColumn(dataset, column, values):
    if (not isinstance(values,list)):
        return (dataset[column] == values)           
    tempMask = (dataset[column] == values[0])
    if (len(values) == 1):
        return tempMask        
    for value in values[1:]:
        tempMask |= (dataset[column] == value)        
    return tempMask

def selectRowsWithSpecificValuesInSpecifiedColumns(dataset,columnsValuesMapping):
    if not len(columnsValuesMapping):
        return dataset
    
    columns, values = list(columnsValuesMapping.keys()), list(columnsValuesMapping.values())
    mask = selectingRowsWithSpecificValuesInSpecifiedColumn(dataset, columns[0],values[0])
    
    if (len(columns) == 1):
        return dataset[mask]
        
    for column, selectedValues in zip(columns[1:], values[1:]):
        tempMask = selectingRowsWithSpecificValuesInSpecifiedColumn(dataset, column, selectedValues)
        mask &= (tempMask)
    return dataset[mask]

def siretTypeChanger(x):
    if x!=x:
        return x
    if len(x.split(' '))!= 1:
        return x
    if len(x.split('.'))!= 1:
        return int(x.split('.')[0] + x.split('.')[1])
    return int(x)
    
    # Merging postal code informations :

def mergePostalCodeValues(x):
    postalCodeSiret = x['codePostalEtablissement']
    postalCodeAssociation = x['CP-Adresse-Code postal']
    if postalCodeSiret == postalCodeSiret:
        return postalCodeSiret
    return postalCodeAssociation

    #Aggregating townships into departments for towns outside of Paris but in the region of IdF :

def aggregateTownship(x):
    township = x['Nom_de_la_commune']
    postal_code = str(x['Code_postal'])
    if postal_code.startswith('77'):
        township='Seine-et-Marne'
    if postal_code.startswith('78'):
        township='Yvelines'
    if postal_code.startswith('91'):
        township='Essonne'
    if postal_code.startswith('92'):
        township='Hauts-de-Seine'
    if postal_code.startswith('93'):
        township='Seine-Saint-Denis'
    if postal_code.startswith('94'):
        township='Val-de-Marne'
    if postal_code.startswith('95'):
        township="Val d'Oise"
    return township


    # Creating data objects : 

def df_to_dict(df, selected_columns):
    tmp_df = df[selected_columns]
    dict = {}
    for idx in range(tmp_df.shape[0]):
        dict[idx] = {}
        for column in selected_columns:
            dict[idx][column]= str(tmp_df.iloc[idx][column])
    return dict

def mapping_properties(property, property_value):
     for key,value in propertyMapper[property].items():
         if (property_value == value):
             return key

def populatingResponse(response, data, properties_order, depth):
    property = properties_order[depth]
    for property_value in data[property].value_counts().index:
        mapped_property_value =mapping_properties(property, property_value)
        response[mapped_property_value] = {}
        tmp_data = data[data[property] == property_value]
        response[mapped_property_value]['total'] = str(tmp_data['subvention_amount'].sum())
        if (len(properties_order) == depth + 1):
            continue
        populatingResponse(response[mapped_property_value], tmp_data, properties_order, depth + 1)

def populatingPerAsso(data, selected_columns):
    res = []
    distinct_val = data['association'].value_counts().index
    for asso in distinct_val:
        res.append({ 'name': asso, 'subventions': df_to_dict(data[data['association'] == asso],selected_columns)}) 
    return res


In [20]:
!curl 'https://files.data.gouv.fr/insee-sirene/StockEtablissement_utf8.zip' > ./data/siret.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1667M  100 1667M    0     0  14.3M      0  0:01:56  0:01:56 --:--:-- 13.9M349M    0     0  20.5M      0  0:01:20  0:00:16  0:01:04 12.6M 1667M   28  482M    0     0  18.5M      0  0:01:29  0:00:25  0:01:04 13.4M 0  16.2M      0  0:01:42  0:00:37  0:01:05 11.9M   0  0:01:44  0:00:40  0:01:04 13.1M   0  14.3M      0  0:01:56  0:01:55  0:00:01 13.6M


In [21]:
!unzip ./data/siret.zip -d data

Archive:  ./data/siret.zip
  inflating: data/StockEtablissement_utf8.csv  


In [4]:
    # Récupération des subventions faites aux associations par la Mairie de Paris :

data_subvention_paris_association = pd.read_csv('https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/subventions-accordees-et-refusees/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B', sep=';', low_memory = False)

    # Récupération de la liste des associations parisiennes :

data_paris_association = pd.read_csv('https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/liste_des_associations_parisiennes/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B', sep=';', low_memory = False)

     # Récupération des codes postaux associés au SIRET :

createFileWithSpecificColumns('./data/StockEtablissement_utf8.csv', './data/postal_code_siret.csv')

data_siret_postal_code = pd.read_csv('./data/postal_code_siret.csv', header = 0, sep = ',')

    # Récupération des données relatives aux codes postaux :

data_postal_code = pd.read_csv('https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/base-officielle-des-codes-postaux/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B', header = 0, sep = ';')[['Code_postal','Nom_de_la_commune']]

In [5]:
#Data preparation :

    # Deleting NaN values :

data_subvention_paris_association = data_subvention_paris_association.dropna(subset = ['Montant voté', 'Année budgétaire'])
data_paris_association=data_paris_association.dropna(subset = ['CP-Adresse-Code postal'])

    # Transforming dtypes :

data_subvention_paris_association = transformOriginDataTypesIntoFinalDataTypes(data_subvention_paris_association, ['float64'], ['int'])
data_paris_association = transformOriginDataTypesIntoFinalDataTypes(data_paris_association, ['float64'], ['int'])
data_subvention_paris_association['Numéro Siret'] = data_subvention_paris_association['Numéro Siret'].apply(siretTypeChanger)
data_postal_code = data_postal_code.astype({ 'Code_postal': str})

    # Merging datasets :

data = pd.merge(data_subvention_paris_association , data_siret_postal_code, how='left', left_on = "Numéro Siret", right_on="siret")
data = pd.merge(data, data_paris_association, how = 'left', left_on = 'Nom Bénéficiaire', right_on = 'PR-Nom Statutaire')

    # Dropping duplicate rows (same file_id) for each year :

minYear = data_subvention_paris_association['Année budgétaire'].min().astype(int)
maxYear = data_subvention_paris_association['Année budgétaire'].max().astype(int)
years = list(range(minYear,maxYear + 1))

final_data = data[data['Année budgétaire'] == minYear].drop_duplicates(subset="Numéro de dossier", keep = 'first')

for year in years[1:]:
    final_data = pd.concat([final_data, data[data['Année budgétaire'] == year].drop_duplicates(subset = "Numéro de dossier", keep="first")])

    # Merging postal code from siret db and association db :

final_data['postal_code'] = final_data.apply(mergePostalCodeValues, axis = 1)

    # Mergin township names with the base data :

final_data = pd.merge(final_data, data_postal_code, how="left", left_on='postal_code', right_on='Code_postal')

    # Aggregating townships into departments for towns outside of Paris but in the region of IdF :

final_data['Nom_de_la_commune'] = final_data.apply(aggregateTownship, axis = 1)

    # Splitting 'Secteurs d'activités définies par l'association' column into 3 different columns :

final_data = pd.concat([final_data, final_data["Secteurs d'activités définies par l'association"].str.split(',', expand = True).rename(columns  = {0: 'activity_1', 1: 'activity_2', 2: 'activity_3'})], axis = 1)
    
    # Selecting columns :

selected_columns = ["Numéro de dossier", 'Année budgétaire', 'Nom Bénéficiaire', 'Objet du dossier', 'Montant voté',
                    'Direction', 'Nature de la subvention', 'activity_1', 'activity_2','activity_3','Code_postal', 'Nom_de_la_commune']

final_data = final_data[selected_columns]

    # Renaming Columns :

mapping_new_names_columns = { "Numéro de dossier": "file_id", 'Année budgétaire':'year', 'Nom Bénéficiaire': 'association',
                             "Objet du dossier": "file_object",'Montant voté': 'subvention_amount', 'Direction': 'city_committee', 
                             "Nature de la subvention": "aim_subvention", "Secteurs d'activités définies par l'association" : "activities", 
                             'Nom_de_la_commune': 'township', 'Code_postal': 'postal_code', 'activity_1': 'activity'}

final_data = final_data.rename(columns = mapping_new_names_columns)

    # Typing as string :

final_data = final_data.astype({'year':str})

    # Transforming nan township values into string 'Inconnue':

final_data['township'] = final_data['township'].fillna('Inconnue')

In [7]:
## Populating aggregated data object :

aggData = {}
non_null = final_data[final_data['subvention_amount'] != 0]
mask = non_null['postal_code'] != non_null['postal_code']

located = non_null[~mask]
unknown = non_null[mask]

aggData['unknown'] = {}
aggData['unknown']['total'] = str(unknown['subvention_amount'].sum())

populatingResponse(aggData, located, ['township', 'year','city_committee', 'activity', 'aim_subvention'], 0)
populatingResponse(aggData['unknown'], unknown, ['year', 'city_committee', 'activity', 'aim_subvention'], 0)

with open('../web/assets/data/data.json', 'w') as f:
    f.write(json.dumps(aggData))

## Populating data per association object :

asso_data = populatingPerAsso(final_data, ['year', 'association', 'subvention_amount', 'city_committee', 'aim_subvention', 'activity', 'township',"file_id", "file_object"])

with open('../web/assets/data/asso_data.json','w') as f :
    f.write(json.dumps(asso_data))